<a href="https://colab.research.google.com/github/KevinTheRainmaker/AI_Tutorials/blob/main/spam_detection_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. TFLite Model Maker 설치

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 5.0 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 237 kB 67.2 MB/s 
     |████████████████████████████████| 3.4 MB 53.7 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 1.2 MB 53.5 MB/s 
     |████████████████████████████████| 6.4 MB 28.4 MB/s 
     |████████████████████████████████| 87 kB 6.0 MB/s 
     |████████████████████████████████| 120 kB 57.7 MB/s 
     |████████████████████████████████| 1.1 MB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.2 MB/s 
     |████████████████████████████████| 840 kB 12.7 MB/s 
     |████████████████████████████████| 25.3 MB 3.3 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 352 kB 56.9 MB/s 
     |████████████████████████████████| 47.8 MB 45 kB/s 
     |████████████████████████████████| 462 kB 52.6 MB/s 
     |████████████████

### 2. 필요 라이브러리 Import

In [14]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/'

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import os
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

### 3. 데이터 불러오기 from Cloud

In [3]:
data_file = tf.keras.utils.get_file(fname='comment-spam.csv',
  origin='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/lmblog_comments.csv',
  extract=False)

245760/235543 [===============================] - 0s 0us/step


In [8]:
pd.read_csv(data_file)

,commenttext,spam
0,still need pick one,False
1,… i similar problem careful version cuda ver...,True
2,visual range five seconds,False
3,family come trusted tonight,False
4,cute,False
...,...,...
995,these principles addition served easy way comp...,True
996,not come just show around help get know place ...,False
997,sure get used,False
998,friend,False


### 4. Pre-Trained 된 Embeddings

In [9]:
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

### 5. 데이터 로더 사용

In [10]:
data = DataLoader.from_csv(
    filename=data_file,
    text_column='commenttext',
    label_column='spam',
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True)

train_data, test_data = data.split(0.9)

### 6. 모델 빌드

In [12]:
model = text_classifier.create(train_data, model_spec=spec, epochs=50,
                               validation_data=test_data)

28/28 [==============================] - 2s 19ms/step - loss: 0.6543 - accuracy: 0.6964 - val_loss: 0.6145 - val_accuracy: 0.7900
Epoch 2/2
28/28 [==============================] - 0s 9ms/step - loss: 0.5946 - accuracy: 0.8002 - val_loss: 0.5472 - val_accuracy: 0.9300
Epoch 3/3
28/28 [==============================] - 0s 11ms/step - loss: 0.5334 - accuracy: 0.8772 - val_loss: 0.4764 - val_accuracy: 0.9600
Epoch 4/4
28/28 [==============================] - 0s 10ms/step - loss: 0.4674 - accuracy: 0.9040 - val_loss: 0.4064 - val_accuracy: 0.9900
Epoch 5/5
28/28 [==============================] - 0s 11ms/step - loss: 0.4077 - accuracy: 0.9330 - val_loss: 0.3413 - val_accuracy: 0.9900
Epoch 6/6
28/28 [==============================] - 0s 9ms/step - loss: 0.3512 - accuracy: 0.9342 - val_loss: 0.2842 - val_accuracy: 0.9800
Epoch 7/7
28/28 [==============================] - 0s 8ms/step - loss: 0.3137 - accuracy: 0.9286 - val_loss: 0.2352 - val_accuracy: 0.9800
Epoch 8/8
28/28 [================

### 7. 완성된 모델 내보내기

In [15]:
model.export(export_dir=path)

In [16]:
# for iOS
model.export(export_dir=path,
             tflite_filename= 'model_ios.tflite',
             export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])